In [1]:
import pandas as pd
import pickle
import numpy as np
pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [2]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

infile = open("model.pickle",'rb')
model = pickle.load(infile)
infile.close()

infile = open("other_info.pickle",'rb')
x = pickle.load(infile)
infile.close()

In [3]:
by_zipcode_dict = x[0]
selected_columns = x[1]

In [21]:
selected_columns

['bedrooms',
 'bathrooms',
 'sqft_living',
 'floors',
 'waterfront',
 'view',
 'sqft_basement',
 'lat',
 'sqft_living15',
 'condition_text_low',
 'grade_text_low',
 'bedrooms^2',
 'bedrooms bathrooms',
 'bedrooms sqft_living',
 'bedrooms sqft_lot',
 'bedrooms floors',
 'bedrooms waterfront',
 'bedrooms view',
 'bedrooms sqft_basement',
 'bedrooms yr_built',
 'bedrooms yr_renovated',
 'bedrooms lat',
 'bedrooms sqft_living15',
 'bedrooms condition_text_low',
 'bedrooms grade_text_low',
 'bathrooms^2',
 'bathrooms sqft_living',
 'bathrooms sqft_lot',
 'bathrooms floors',
 'bathrooms waterfront',
 'bathrooms view',
 'bathrooms sqft_basement',
 'bathrooms yr_built',
 'bathrooms yr_renovated',
 'bathrooms lat',
 'bathrooms sqft_living15',
 'bathrooms sqft_lot15',
 'bathrooms condition_text_low',
 'bathrooms condition_text_mid',
 'bathrooms grade_text_low',
 'sqft_living^2',
 'sqft_living sqft_lot',
 'sqft_living floors',
 'sqft_living waterfront',
 'sqft_living view',
 'sqft_living sqft_bas

In [22]:
model.coef_

array([-1.08006250e+06, -8.28327567e+05,  5.89202706e+03, -1.86609923e+06,
       -3.29708632e+07, -1.30872012e+06, -3.19173420e+03,  5.72311675e+07,
        3.21291051e+03, -5.00980863e+01, -5.78881796e+04, -1.29174544e+05,
       -2.05058188e+05,  2.01586785e+03, -3.43099570e+03, -2.40289692e+01,
        6.91568291e-02,  2.21493051e+04,  6.48398392e+04, -8.79975823e+03,
        1.40392902e+01,  3.78006590e+02, -8.77870219e+00,  8.13329380e+03,
       -1.52296853e+00, -6.23800303e-02, -3.87602446e+02,  1.66282045e+04,
       -5.07883882e+00, -7.38227875e-02,  6.17347699e+03,  1.16210712e+05,
       -3.31125338e+03,  2.82137276e+01,  1.32714703e+03,  3.69024490e+01,
       -3.94700125e+04, -1.37835236e+01,  3.46189259e-01,  5.46353578e-03,
        1.64734326e+03,  1.36244839e+02,  3.86305153e-03,  3.52878124e-05,
       -2.47764509e+01,  1.23021627e+02,  1.43148108e+01, -1.91179382e-02,
       -3.18544086e+00, -3.66525219e-02,  1.13858345e+01,  8.13720608e-03,
       -1.92152336e-04,  

In [ ]:
# final_scaler = read_pickle(filename)
# final_model = read_pickle(filename)

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [4]:
holdout['date_formatted'] = pd.to_datetime(holdout['date'])

In [5]:
holdout['sqft_living']= np.where(holdout['sqft_living'] >8000, 8000, holdout['sqft_living'])
holdout['sqft_lot']= np.where(holdout['sqft_lot'] >800000, 800000, holdout['sqft_lot'])
holdout['sqft_basement']= np.where(holdout['sqft_basement'] >2000, 2000, holdout['sqft_basement'])

In [6]:
holdout['yr_sold']= holdout['date_formatted'].dt.year
holdout['yr_since_update']= np.where(holdout['yr_renovated'] ==0, holdout['yr_sold']-holdout['yr_built'], holdout['yr_sold']-holdout['yr_renovated'])
holdout['yr_built_abs'] = abs(1958-holdout['yr_built'])

In [7]:
holdout['bedrooms']= np.where(holdout['bedrooms'] <1 , holdout['floors']-1, holdout['bedrooms'])
holdout['bathrooms']= np.where(holdout['bathrooms'] <1, holdout['floors'], holdout['bathrooms'])
holdout['has_basement']= np.where(holdout['sqft_basement'] >0, 1, 0)
holdout['bedrooms_5plus']= np.where(holdout['bedrooms'] >4, 1, 0)
holdout['bedrooms_2.5minus']= np.where(holdout['bedrooms'] <3, 1, 0)
holdout['bathrooms_2.75plus']= np.where(holdout['bathrooms'] >2.5, 1, 0)

In [8]:
holdout['condition_text']=holdout['condition']
holdout['condition_text']=np.where(holdout['condition']<3, 'low', holdout['condition_text'])
holdout['condition_text']=np.where(holdout['condition']==3, 'mid', holdout['condition_text'])
holdout['condition_text']=np.where(holdout['condition']>3, 'high', holdout['condition_text'])

In [9]:
holdout['grade_text']=holdout['grade']
holdout['grade_text']=np.where(holdout['grade']<6, 'low', holdout['grade_text'])
holdout['grade_text']=np.where((holdout['grade']<8)&(holdout['grade']>5), 'mid_1', holdout['grade_text'])
holdout['grade_text']=np.where((holdout['grade']<10)&(holdout['grade']>7), 'mid_2', holdout['grade_text'])
holdout['grade_text']=np.where(holdout['grade']>9, 'high', holdout['grade_text'])

In [10]:
holdout['zipcode_text']=holdout['zipcode']
for zipcode in [98002, 98168, 98032, 98148, 98001, 98188, 98023, 98003, 98030,98031]:
    holdout['zipcode_text']=np.where(holdout['zipcode']==zipcode , 'n_1', holdout['zipcode_text'])
for zipcode in [98198, 98055, 98022, 98042, 98178, 98106, 98092, 98058, 98108,98146]:
    holdout['zipcode_text']=np.where(holdout['zipcode']==zipcode , 'n_2', holdout['zipcode_text'])
for zipcode in [98038, 98133, 98118, 98155, 98056, 98010, 98126, 98019, 98045,98014]:
    holdout['zipcode_text']=np.where(holdout['zipcode']==zipcode , 'n_3', holdout['zipcode_text'])
for zipcode in [98028, 98125, 98166, 98011, 98059, 98070, 98034, 98065, 98136,98072]:
    holdout['zipcode_text']=np.where(holdout['zipcode']==zipcode , 'n_4', holdout['zipcode_text'])
for zipcode in [98117, 98107, 98103, 98024, 98007, 98027, 98115, 98116, 98029,98122]:
    holdout['zipcode_text']=np.where(holdout['zipcode']==zipcode , 'n_5', holdout['zipcode_text'])
for zipcode in [98144, 98008, 98052, 98177, 98077, 98053, 98074, 98199, 98075,98033]:
    holdout['zipcode_text']=np.where(holdout['zipcode']==zipcode , 'n_6', holdout['zipcode_text'])
for zipcode in [98005, 98006, 98105, 98119, 98109, 98102, 98112, 98040, 98004,98039]:
    holdout['zipcode_text']=np.where(holdout['zipcode']==zipcode , 'n_7', holdout['zipcode_text'])

In [11]:
holdout = pd.get_dummies(holdout, columns=['condition_text', 'grade_text', 'zipcode_text'], drop_first=True)

In [12]:
holdout['for_zipcode'] = [tuple(x) for x in holdout[['zipcode','bedrooms_5plus','bedrooms_2.5minus','bathrooms_2.75plus']].to_numpy()]

In [13]:
holdout['comp'] = np.nan
holdout['comp'].fillna(holdout['for_zipcode'].map(by_zipcode_dict), inplace=True)

In [15]:
# From F-test: ['condition','zipcode', 'long', 'condition_text_mid', 'zipcode_text_n_4', 'yr_sold']
# dropped 'sqft_above','condition','zipcode', 'long','yr_sold'
features_3=['bedrooms', 'bathrooms', 'sqft_living',  # excl id and bedrooms and 'date', 
       'sqft_lot', 'floors', 'waterfront', 'view', #'grade',  'condition', 
       'sqft_basement', 'yr_built', 'yr_renovated', # 'zipcode',
       'lat', 'sqft_living15', 'sqft_lot15', #'date_formatted', 'long', 
       'condition_text_low', 'condition_text_mid', 'grade_text_low',
       'grade_text_mid_1', 'grade_text_mid_2', 'has_basement',
       'zipcode_text_n_2', 'zipcode_text_n_3', 'zipcode_text_n_4',
       'zipcode_text_n_5', 'zipcode_text_n_6', 'zipcode_text_n_7',
       'bedrooms_5plus', 'bathrooms_2.75plus', 'comp', 'bedrooms_2.5minus',
       'yr_since_update', 'yr_built_abs'] # 'for_zipcode', 'yr_sold', 

df_features_3 = holdout[features_3]

In [16]:
contin_vars = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'sqft_basement', 'yr_built', # 'condition', 'grade', 
       'yr_renovated', 'lat', 'sqft_living15', 'sqft_lot15', #'zipcode', 'long', 
#        'condition_text_low', 'condition_text_mid', 'grade_text_low',
#        'has_basement', 'grade_text_mid_1', 'grade_text_mid_2', 
#        'zipcode_text_n_2', 'zipcode_text_n_3', 'zipcode_text_n_4',
#        'zipcode_text_n_5', 'zipcode_text_n_6', 'zipcode_text_n_7',
       'comp', # 'bedrooms_5plus', 'bathrooms_2.75plus', 'bedrooms_2.5minus',
       'yr_since_update', 'yr_built_abs'] # 'yr_sold', 

In [18]:
from sklearn.preprocessing import PolynomialFeatures

In [19]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(df_features_3[contin_vars])
poly2_columns = poly_2.get_feature_names(df_features_3.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
df_poly2.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,sqft_basement,yr_built,yr_renovated,lat,sqft_living15,sqft_lot15,condition_text_low,condition_text_mid,grade_text_low,bedrooms^2,bedrooms bathrooms,bedrooms sqft_living,bedrooms sqft_lot,bedrooms floors,bedrooms waterfront,bedrooms view,bedrooms sqft_basement,bedrooms yr_built,bedrooms yr_renovated,bedrooms lat,bedrooms sqft_living15,bedrooms sqft_lot15,bedrooms condition_text_low,bedrooms condition_text_mid,bedrooms grade_text_low,bathrooms^2,bathrooms sqft_living,bathrooms sqft_lot,bathrooms floors,bathrooms waterfront,bathrooms view,bathrooms sqft_basement,bathrooms yr_built,bathrooms yr_renovated,bathrooms lat,bathrooms sqft_living15,bathrooms sqft_lot15,bathrooms condition_text_low,bathrooms condition_text_mid,bathrooms grade_text_low,sqft_living^2,sqft_living sqft_lot,sqft_living floors,sqft_living waterfront,sqft_living view,sqft_living sqft_basement,sqft_living yr_built,sqft_living yr_renovated,sqft_living lat,sqft_living sqft_living15,sqft_living sqft_lot15,sqft_living condition_text_low,sqft_living condition_text_mid,sqft_living grade_text_low,sqft_lot^2,sqft_lot floors,sqft_lot waterfront,sqft_lot view,sqft_lot sqft_basement,sqft_lot yr_built,sqft_lot yr_renovated,sqft_lot lat,sqft_lot sqft_living15,sqft_lot sqft_lot15,sqft_lot condition_text_low,sqft_lot condition_text_mid,sqft_lot grade_text_low,floors^2,floors waterfront,floors view,floors sqft_basement,floors yr_built,floors yr_renovated,floors lat,floors sqft_living15,floors sqft_lot15,floors condition_text_low,floors condition_text_mid,floors grade_text_low,waterfront^2,waterfront view,waterfront sqft_basement,waterfront yr_built,waterfront yr_renovated,waterfront lat,waterfront sqft_living15,waterfront sqft_lot15,waterfront condition_text_low,waterfront condition_text_mid,waterfront grade_text_low,view^2,view sqft_basement,view yr_built,view yr_renovated,view lat,view sqft_living15,view sqft_lot15,view condition_text_low,view condition_text_mid,view grade_text_low,sqft_basement^2,sqft_basement yr_built,sqft_basement yr_renovated,sqft_basement lat,sqft_basement sqft_living15,sqft_basement sqft_lot15,sqft_basement condition_text_low,sqft_basement condition_text_mid,sqft_basement grade_text_low,yr_built^2,yr_built yr_renovated,yr_built lat,yr_built sqft_living15,yr_built sqft_lot15,yr_built condition_text_low,yr_built condition_text_mid,yr_built grade_text_low,yr_renovated^2,yr_renovated lat,yr_renovated sqft_living15,yr_renovated sqft_lot15,yr_renovated condition_text_low,yr_renovated condition_text_mid,yr_renovated grade_text_low,lat^2,lat sqft_living15,lat sqft_lot15,lat condition_text_low,lat condition_text_mid,lat grade_text_low,sqft_living15^2,sqft_living15 sqft_lot15,sqft_living15 condition_text_low,sqft_living15 condition_text_mid,sqft_living15 grade_text_low,sqft_lot15^2,sqft_lot15 condition_text_low,sqft_lot15 condition_text_mid,sqft_lot15 grade_text_low,condition_text_low^2,condition_text_low condition_text_mid,condition_text_low grade_text_low,condition_text_mid^2,condition_text_mid grade_text_low,grade_text_low^2
0,4.0,2.50,2270.0,11500.0,1.0,0.0,0.0,730.0,1967.0,0.0,47.7089,2020.0,10918.0,469453.345794,47.0,9.0,16.0,10.00,9080.0,46000.0,4.0,0.0,0.0,2920.0,7868.0,0.0,190.8356,8080.0,43672.0,1.877813e+06,188.0,36.0,6.2500,5675.0,28750.0,2.50,0.0,0.0,1825.0,4917.50,0.0,119.272250,5050.0,27295.00,1.173633e+06,117.5,22.50,5152900.0,26105000.0,2270.0,0.0,0.0,1657100.0,4465090.0,0.0,108299.203,4585400.0,24783860.0,1.065659e+09,106690.0,20430.0,132250000.0,11500.0,0.0,0.0,8395000.0,22620500.0,0.0,548652.3500,23230000.0,125557000.0,5.398713e+09,540500.0,103500.0,1.0,0.0,0.0,730.0,1967.0,0.0,47.7089,2020.0,10918.0,4.694533e+05,47.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,532900.0,1435910.0,0.0,34827.497,1474600.0,7970140.0,3.427009e+08,34310.0,6570.0,3869089.0,0.0,93843.4063,3973340.0,21475706.0,9.234147e+08,92449.0,17703.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [20]:
cat_vars = [x for x in df_features_3.columns if x not in contin_vars]
df_poly_cat = pd.concat([df_poly2, df_features_3[cat_vars]], axis=1)

In [ ]:
# transformed_holdout = final_scaler(holdout)

## Step 3: Predict the holdout set

In [24]:
final_answers = model.predict(df_poly_cat[selected_columns])

In [30]:
final_answers[:100]

array([ 544856.89862323,  545733.02364779,  487922.12532258,
        447850.95627427,  622277.62322974,  656440.01609206,
        332835.27754998,  362153.18412614,  470470.391644  ,
        342323.60672617,  714309.16991758,  432470.16654086,
        435256.17866039,  243338.82339811,  410659.26962543,
        745123.91581607,  762619.32905698,  608768.52761817,
        652218.47983146,  191856.1912899 ,  650088.20150924,
        489533.30206919, 1251456.06453276,  229234.69754505,
        266165.23349166,  309747.74793506,  559078.67786264,
        471304.2506628 ,  563532.01056147,  536326.23748899,
        582196.33537078,  455976.52100706,  451739.39901304,
        392542.22840691, 1299981.45723557,  918867.03564334,
        169518.44509792,  575318.8621912 , 1516122.34250259,
        209301.41015625,  354429.90449476,  547379.56428909,
        334175.76578903,  477029.55411077,  482877.65440559,
        509769.88117552,  722334.44932413,  639127.63809633,
        818702.19963169,

## Step 4: Export your predictions

In [33]:
# final_answers.to_csv('housing_preds_D_Kokenova.csv')

In [32]:
# prediction = pd.DataFrame(final_answers, columns=['predictions']).to_csv('housing_preds_D_Kokenova.csv')
pd.DataFrame(final_answers, columns=['predictions']).to_csv('housing_preds_D_Kokenova.csv')